# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[27744, 27745, 27746, 27747, 27748, 27749, 27804]

In [5]:
ifps = {id: IFP(id) for id in today_ids}

In [6]:
from Forecaster import Forecaster

In [7]:
fcst = Forecaster(ifps)

In [8]:
fcst.fit()

MODEL DOMAIN
27744 Will the Russian 2024 autumn draft demobilization start on time?
State executive action
27745 Will Taylor Swift publicly endorse Kamala Harris for president before October 1, 2024?
Election outcome
27746 Will Magnus Carlsen win the Speed Chess Championship 2024?
Sports performance
27747 Will Hikaru Nakamura win the Speed Chess Championship 2024?
Sports performance
27748 Will Hans Niemann win the Speed Chess Championship 2024?
Sports performance
27749 Will Alireza Firouzja win the Speed Chess Championship 2024?
Sports performance
27804 Will Pavel Durov leave France before October 1, 2024?
Leadership change
CORRELATOR
|Russian 2024 autumn draft demobilization|27744|Will the Russian 2024 autumn draft demobilization start on time?|
|Taylor Swift endorsement|27745|Will Taylor Swift publicly endorse Kamala Harris for president before October 1, 2024?|
|Speed Chess Championship 2024|27746|Will Magnus Carlsen win the Speed Chess Championship 2024?|
|Speed Chess Championship 

In [9]:
fcst.report()

27744 Will the Russian 2024 autumn draft demobilization start on time?
Forecast 85
Rationale The historical trend of timely publication of presidential decrees for demobilization, usually in late September, strongly suggests that the 2024 autumn draft demobilization will start on time. The regularity and predictability of this process, designed to ensure an orderly transition between conscription cycles, supports a high probability that a decree will be issued on time, specifying that demobilization starts by October 7, 2024. Additionally, the recent legislative and administrative measures to enhance the drafting and tracking of conscripts indicate preparedness and adherence to schedules. Therefore, I assign a probability of 85%. 

27745 Will Taylor Swift publicly endorse Kamala Harris for president before October 1, 2024?
Forecast 70
Rationale Several sources close to Taylor Swift suggest she is preparing to endorse Kamala Harris in late September or early October, within the stated t

In [10]:
fcst.upload()

Prediction posted for  27744
Comment posted for  27744
Prediction posted for  27745
Comment posted for  27745
Prediction posted for  27746
Comment posted for  27746
Prediction posted for  27747
Comment posted for  27747
Prediction posted for  27748
Comment posted for  27748
Prediction posted for  27749
Comment posted for  27749
Prediction posted for  27804
Comment posted for  27804
